In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
#Datene einlesen
data = pd.read_csv('./data/data.csv',usecols = ['Document Title','Abstract','Publication Year'])
data.columns = ['title','year','abstract']

data.head()

,title,year,abstract
0,Is an Athletic Approach the Future of Software...,2016,Traditional software engineering education app...
1,What Do We Know about Knowledge Management? Pr...,2009,There have been many claims about knowledge ma...
2,Global Software Engineering: An Industry Persp...,2016,Professional software products and IT systems ...
3,Advancing Software Engineering Professional Ed...,2011,The paper mentions that a reference curriculum...
4,Improving the State of Automotive Software Eng...,2017,The automotive industry is fundamentally chang...


In [4]:
data.columns


Index(['title', 'year', 'abstract'], dtype='object')

In [5]:
data.count()

title       2292
year        2292
abstract    2276
dtype: int64

## Funktion für die TermDokumentenMatrix

In [6]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (2,2),dtype=np.int32)
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), dtype=np.int32)
    df['year'] = data['year']
    returndf = df.groupby('year').sum().transpose().astype('int32')
    return returndf

In [7]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [8]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

## Erstellen der TermDokumentenmatrix

In [9]:

#abstract = CreateTermDocMatrix('abstract')

In [10]:
#b = abstract.loc[abstract[2009] > 15] 

In [11]:
#b.plot.bar()
#print(b)

In [12]:
from collections import Counter
#cnt = Counter()

#for abstract in data['Abstract']:
#    if type(abstract) is str:
#        wordlist = abstract.split(' ')
#        for word in wordlist:
#            cnt[word] += 1
        
#cnt.most_common(30)

In [13]:
#data[2019].count()

In [14]:
#for abstract in data.itertuples():
#    print(abstract[1])
#    print(abstract[2])
#    print(abstract[3])

In [15]:
#Untersuchung Title
#title = CreateTermDocMatrix('abstract')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 10] 

#b.plot.bar()
#print(b)
#b.shape

In [16]:
#import spacy
#from spacy.symbols import NOUN
#from collections import Counter


#nlp = spacy.load("en_core_web_sm")

#cnt = Counter()
#tokenlist = []
#i = 1

#for columns in data.itertuples():

#    text = columns[1] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
#    if type(text) is str:
#        doc = nlp(text)

#        for noun_phrase in list(doc.noun_chunks):
#             noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)

#        [(token.text,token.pos_) for token in doc]

In [17]:
from matplotlib import pyplot as plt
years = []

#Balkendiagramm
#plt.bar(range(len(years)),anzah)

#Titel
#plt.title("Toller Plot")
#plt.ylabel("# of awards")

#plt.xticks(range(len(years)),years)
#plt.show()

In [18]:
#def filter_spans(span):
#    return span

## Module importieren

In [19]:
import spacy
from spacy.symbols import NOUN
from spacy.matcher import Matcher
from collections import Counter
from spacy.util import filter_spans

## NLP Model laden

In [20]:
nlp = spacy.load("en_core_web_sm")

## Counter und Matcher initialisieren

In [21]:
matcher = Matcher(nlp.vocab)
words = Counter()
words_chunk = Counter()

## Pattern festlegen und dem Matcher hinzufügen

In [22]:
pattern_test = [{'POS': 'NOUN'},
                {'POS': 'NOUN'}]

pattern_exmaple = [{'POS': 'ADJ', 'OP': '?'},
                   {'OP': '+', 'POS': 'NOUN'},
                   {'POS': 'NOUN', 'OP': '?'}]

In [23]:
matcher.add("match_test", [pattern_test])
matcher.add("match_example", [pattern_exmaple])

In [24]:
for columns in data.itertuples():
    
    text = columns[3] #1 = title 2 = year 3 = abstract
    # Verarbeite den Text
    if type(text) is str:
        doc = nlp(text)


    matches = matcher(doc)
    sp  = [doc[match[1]:match[2]] for match in matches] 

    filtered = filter_spans(sp)
    with doc.retokenize() as retokenizer:
        for s in filtered:
            retokenizer.merge(s, attrs = {"POS": "NOUN"})
    [(token.pos_, token.text) for token in doc]
        
    for token in doc:
        # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        token_ent = token.ent_type_


        if token.pos == NOUN:
            if not token.is_stop:
                words[token_text] += 1

In [25]:
words.most_common(100)

[('paper', 712),
 ('approach', 634),
 ('developers', 580),
 ('results', 458),
 ('software', 365),
 ('code', 317),
 ('article', 314),
 ('percent', 279),
 ('system', 267),
 ('model', 266),
 ('time', 259),
 ('%', 256),
 ('techniques', 241),
 ('data', 235),
 ('systems', 234),
 ('state', 208),
 ('set', 207),
 ('software engineering', 198),
 ('number', 193),
 ('performance', 192),
 ('study', 188),
 ('analysis', 185),
 ('tools', 185),
 ('work', 181),
 ('problem', 177),
 ('effectiveness', 171),
 ('framework', 171),
 ('models', 171),
 ('development', 169),
 ('terms', 169),
 ('method', 167),
 ('researchers', 163),
 ('approaches', 162),
 ('projects', 161),
 ('users', 160),
 ('use', 158),
 ('quality', 153),
 ('process', 152),
 ('practitioners', 151),
 ('application', 149),
 ('applications', 148),
 ('technique', 148),
 ('tool', 147),
 ('context', 144),
 ('research', 143),
 ('challenges', 138),
 ('requirements', 138),
 ('methods', 132),
 ('order', 130),
 ('issue', 129),
 ('programs', 128),
 ('testin